In [7]:
!pip install openmeteo_requests
!pip install requests_cache
!pip install pmdarima
!pip install retry_requests
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from retry_requests import retry

In [4]:
model_df=pd.read_csv("model_df.csv")
model_df=model_df.set_index("date",drop=True)
column_n=model_df.columns.get_loc("Humedad_rel")
model_df.head()

,Temperatura,Humedad_rel,Presion_mar,Presion_sup,et0_Evot,Deficit_VP,year,month,day,hour,...,retardado_30_Presion_mar,retardado_30_Presion_sup,retardado_30_et0_Evot,retardado_30_Deficit_VP,retardado_365_Temperatura,retardado_365_Humedad_rel,retardado_365_Presion_mar,retardado_365_Presion_sup,retardado_365_et0_Evot,retardado_365_Deficit_VP
date,,,,,,,,,,,,,,,,,,,,,
1960-12-31 03:00:00,25.332499,80.98457,1011.2,1010.50580,0.068116,0.614003,1960,12,31,3,...,1009.9,1009.20920,0.058346,0.563104,25.582499,87.398070,1012.4,1011.70570,0.044125,0.412929
1960-12-31 04:00:00,25.282500,81.97270,1011.0,1010.30590,0.062004,0.580363,1960,12,31,4,...,1010.1,1009.40880,0.053853,0.550115,25.432499,87.649290,1012.1,1011.40550,0.040805,0.401119
1960-12-31 05:00:00,25.232500,82.71973,1010.9,1010.20575,0.057649,0.554659,1960,12,31,5,...,1009.4,1008.70860,0.048444,0.527305,25.332499,87.376040,1011.8,1011.10535,0.039934,0.407574
1960-12-31 06:00:00,25.232500,82.71973,1010.3,1009.60614,0.056459,0.554659,1960,12,31,6,...,1008.9,1008.20870,0.045948,0.504812,25.182499,87.893230,1011.7,1011.00520,0.036853,0.387411
1960-12-31 07:00:00,25.182499,83.47328,1010.4,1009.70610,0.053528,0.528892,1960,12,31,7,...,1008.7,1008.00885,0.044298,0.461862,24.932499,88.943016,1010.9,1010.20500,0.031484,0.348597


In [5]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(model_df)

# Define sequence length and features
sequence_length = 24  # Number of time steps in each sequence
num_features = len(model_df.columns)

# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i+sequence_length]
    label = scaled_data[i+sequence_length][column_n]  # 'Humedad' column index
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

# Split into train and test sets
train_size = int(0.8 * len(sequences))
rain_train_x, rain_test_x = sequences[:train_size], sequences[train_size:]
rain_train_y, rain_test_y = labels[:train_size], labels[train_size:]

print("Model_df",model_df.shape)

print("Train X shape:", rain_train_x.shape)
print("Train Y shape:", rain_train_y.shape)

print("Test X shape:", rain_test_x.shape)
print("Test Y shape:", rain_test_y.shape)

Model_df (552288, 29)
Train X shape: (441811, 24, 29)
Train Y shape: (441811,)
Test X shape: (110453, 24, 29)
Test Y shape: (110453,)


In [6]:
model_pkl_file = "modelo_temperatura.pkl"

with open(model_pkl_file, 'rb') as file:  
    rain_model = pickle.load(file)

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming you have trained the model and have the 'best_model' object
# Also, 'test_x' and 'test_y' should be available
print(" ---------- Cáclulo de métricas de rendimiento ---------- ")
# Predict temperatures using the trained model
predictions = rain_model.predict(rain_test_x)

# Calculate evaluation metrics
mae = mean_absolute_error(rain_test_y, predictions)
mse = mean_squared_error(rain_test_y, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

 ---------- Cáclulo de métricas de rendimiento ---------- 


ValueError: in user code:

    File "C:\Users\Isaác Mauricio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Isaác Mauricio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Isaác Mauricio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Isaác Mauricio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-pack

In [ ]:
future_hours=90*24
row_count=len(model_df.columns)
# y_true values
test_y_copies = np.repeat(rain_test_y.reshape(-1, 1), rain_test_x.shape[-1], axis=-1)
true_temp = scaler.inverse_transform(test_y_copies)[:,column_n]

# predicted values
prediction = rain_model.predict(rain_test_x)
prediction_copies = np.repeat(prediction, row_count, axis=-1)
predicted_temp = scaler.inverse_transform(prediction_copies)[:,column_n]

# predicted future values
prediction = rain_model.predict(rain_test_x[-future_hours:])
prediction_copies = np.repeat(prediction, row_count, axis=-1)
predicted_future_temp = scaler.inverse_transform(prediction_copies)[:,column_n]

#save as csv
#predicted_temp.to_csv("Predicion_temperatura")
#model_df.to_csv("Model_df")

In [ ]:
predict_period_dates = pd.date_range(list(model_df.index)[-1], periods=future_hours, freq="h")
print(predict_period_dates)
df_forecast=pd.DataFrame({"date":predict_period_dates,"Humedad_rel":predicted_future_temp})
df_forecast["date"]=pd.to_datetime(df_forecast["date"])
df_forecast=df_forecast.set_index("date", drop=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(model_df["Humedad_rel"].index[-77:], true_temp[-77:], label='Real')
plt.plot(model_df["Humedad_rel"].index[-77:], predicted_temp[-77:], label='Pronóstico')
plt.title('Pronóstico de Temperatura vs Valores reales')
plt.xlabel('Tiempo')
plt.ylabel('Temperatura')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(model_df["Humedad_rel"][-77:], label='Real')
plt.plot(df_forecast[0:150], label='Pronóstico')
plt.title('Pronóstico de Humedad relativa futura')
plt.xlabel('Tiempo')
plt.ylabel('Humedad Relativa')
plt.legend()
plt.show()